<a href="https://colab.research.google.com/github/AndreaDvorakova/Heureka_Scraping/blob/main/Heureka_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [36]:
#requesting html code from Heureka with paddleboards
response = requests.get('https://paddleboardy.heureka.cz/f:24551:41990821/')
soup = BeautifulSoup(response.text, 'html.parser')

#finding all the product blocks in html based on h3 tag
products = soup.find_all('h3', {'class':'c-product__title'})

#finding all the titles and related links in html 
titles = [product.find('a').text for product in products]
hrefs = [product.find('a')['href'] for product in products]
all_products = []

#iterating through titles and hrefs to get into the detail of each product and find in which store is selling it
for href, title in zip(hrefs, titles):
  response2 = requests.get(href)
  soup2 = BeautifulSoup(response2.text, 'html.parser')
  
  #picking from the html the shops which are selling the product
  shops = soup2.find_all('div', {'class':'c-offer__inner'})

  offers_list = []
  #finding the price of the product in the shop and transferring into clear int variable
  for shop in shops:
    price = [letter.replace('\xa0', '') for letter in (shop.find('span', {'class':'c-offer__price u-bold u-delta'}).text[:-2])]

    s = [str(integer) for integer in price]
    a_string = "".join(s)
    res = int(a_string)
    
    try:
      #creating and offer with title, price and shop
      offer = {
          'title': title,
          'price': res,
          'shop': shop.find('img', {'class':'c-offer__shop-logo e-image-with-fallback'})['alt'],
          'shop_link': shop.find('a', {'class': 'c-offer__shop-logo-cont'})['href'],
          'shipping_cost' : shop.find('div', {'class':'c-offer__delivery-availability'})
          }
      offers_list.append(offer)

    except Exception as e:
      offers_list.append({})

  #list of all products
  all_products = all_products + offers_list

#transferring list of all products into dataframe
df = pd.DataFrame(all_products) 

#replacing None for zero and changing data types
df = df.replace({'shipping_cost': {None:'0'}})
df = df.dropna()
df = df.astype({
    'shipping_cost':'int32',
    'price':'int32'
})

#sum of price and shipping cost as a total price
df['total_price'] = df['price'] + df['shipping_cost']
print(df)

                            title  price                 shop  \
0  Paddleboard Coasto Evasion 7'4  12349       Snowboardel.cz   
1  Paddleboard Coasto Evasion 7'4  11250      ONLINE-SPORT.cz   
2  Paddleboard Coasto Evasion 7'4  11699     HEADsportSHOP.cz   
3  Paddleboard Coasto Evasion 7'4  12349              MALL.CZ   
4  Paddleboard Coasto Evasion 7'4  12999     Shotboardshop.cz   
5  Paddleboard Coasto Evasion 7'4  13126         scootshop.cz   
6    Paddleboard Tambo YOGA 10’5″  23900           E-čluny.cz   
7    Paddleboard Tambo YOGA 10’5″  23900  Nafukovací čluny.cz   
8    Paddleboard Tambo YOGA 10’5″  23900        VodackyRaj.cz   

                                           shop_link  shipping_cost  \
0  https://www.heureka.cz/exit/shop-snowboardel-c...              0   
1  https://www.heureka.cz/exit/online-sport-cz/42...              0   
2  https://www.heureka.cz/exit/headsportshop-cz/4...              0   
3  https://www.heureka.cz/exit/mall-cz/4630314089...             